# Building QA system with two different LLMs including Llama and Mistral

Intro: We build a QA system managed by the manager agent. Manager agent can obtain questions from human input and assign the question to two different large language models LLama and Mixtral.


In [3]:
# add related imports here
from agentlite.agents import ABCAgent
from agentlite.llm.agent_llms import get_llm_backend
from agentlite.llm.LLMConfig import LLMConfig
from agentlite.logging.multi_agent_log import AgentLogger
from env import openai_api_key
from agentlite.commons import AgentAct, TaskPackage
from agentlite.actions import ThinkAct, FinishAct
from agentlite.actions.InnerActions import INNER_ACT_KEY
from agentlite.agents.agent_utils import AGENT_CALL_ARG_KEY
import openai

## Using fastchat to host Llama and Mixtral on your own server

Run the following commands in terminal

1. Install fastchat:
```
    pip3 install "fschat[model_worker,webui]"
```
2. Open a new terminal to serve fastchat.serve.controller:
```
    python3 -m fastchat.serve.controller --host 0.0.0.0
```
3. Open a new terminal to serve llama (remember to get your access on huggingface to llama):
```
    CUDA_VISIBLE_DEVICES=0 python3 -m fastchat.serve.model_worker --model-names "llama" --model-path meta-llama/Llama-2-7b-chat-hf --host 0.0.0.0  --port 31001 --worker http://localhost:31001 --max-gpu-memory 15000MIB --limit-worker-concurrency 16 --num-gpus 1
```
4. Open a new terminal to serve Mixtral
```
    CUDA_VISIBLE_DEVICES=1 python3 -m fastchat.serve.model_wor
ker --model-names "mixtral" --model-path mistralai/Mistral-7B-Instruct-v0.2 --host 0.0.0.0  --port 31003 --worker http://localhost:31003 --max-gpu-memory 15000MIB --limit-worker-concurrency 16 --num-gpus 1 
```
5. Open a new terminal to serve fastchat openai api
```
python3 -m fastchat.serve.openai_api_server --host 0.0.0.0 --port 8000
```

## Define and initialize the agent

In [4]:
# Define the LLM agent
class LLMAgent(ABCAgent):
    def __init__(self, name):
        role = f"""You are a large language model named {name}, and you can answer questions given by user."""
        self.name = name
        self.temperature = 0.5
        self.stop = ['\n']
        super().__init__(
            name=self.name,
            role=role,
        ) 
    
    def __call__(self, question: TaskPackage):
        openai.api_key = "EMPTY"  # Not support yet
        openai.base_url = "http://localhost:8000/v1/"
        prompt = question.instruction + " " + "[/INST]" # Add end tokens for the prompt

        completion = openai.completions.create(
                    model=self.name,
                    temperature=self.temperature,
                    stop=self.stop,
                    prompt=prompt,
                    max_tokens=128, #self.max_tokens,
                )
    
        output = completion.choices[0].text
        return output

# Initialize logger
agent_logger = AgentLogger(PROMPT_DEBUG_FLAG=False)

llama = LLMAgent("llama")
mixtral = LLMAgent("mixtral")



In [5]:
# Define the human agent
class Human(ABCAgent):
    def __init__(self):
        name = "Human"
        role = """You are human to give questions to all the other agents. You can ask questions to Llama Mixtral."""
        super().__init__(
            name=name,
            role=role,
        ) 
    
    def __call__(self, question):
        print(question)
        print("Please input your instruction here: ")
        instruction = input()

        return f"My instruction is: {instruction}"
    
# Initialize human agent
human_agent = Human()

In [6]:
# define manager agent
from agentlite.agents import ManagerAgent

llm_config_dict = {"llm_name": "gpt-4-32k", "temperature": 0.0, "openai_api_key": openai_api_key}
llm_config = LLMConfig(llm_config_dict)
llm = get_llm_backend(llm_config)


# Initialize manager agent
manager_agent_info = {
    "name": "manager_agent",
    "role": "You are controlling llama, mixtral and the human_agent. Each time you need to acquire a question from the human_agent, and you should decide which agent to answer the question."}
team = [llama, mixtral, human_agent]
manager_agent = ManagerAgent(name=manager_agent_info["name"], role=manager_agent_info["role"], llm=llm, TeamAgents=team,logger=agent_logger)

## Add an illustration example for the manager agent

In [7]:
# Add illustration example for the manager agent
exp_task = "Follow instructions from human to ask questions on different domains"
exp_task_pack = TaskPackage(instruction=exp_task)


act_1 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""I need to get a question from human first."""
    },
)
obs_1 = "OK"


act_2 = AgentAct(
    name=human_agent.name,
    params={AGENT_CALL_ARG_KEY: "Do you have any questions?",
        },
)
obs_2 = """I want to ask llama model who is the best large language model?"""


act_3 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""It is a question for llama. I need to ask the llama to answer the question."""
    },
)
obs_3 = "OK"


act_4 = AgentAct(
    name=llama.name,
    params={AGENT_CALL_ARG_KEY: "Who is the best large language model?",
        },
)
obs_4 = """Apparent Llama is the best one."""


act_5 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""I have answered the question from human. I need to collect the next question from human."""
    },
)
obs_5 = "OK"


act_6 = AgentAct(
    name=human_agent.name,
    params={AGENT_CALL_ARG_KEY: "Do you have another questions?",
        },
)
obs_6 = """Who is the best large language model? I want to ask mixtral model."""

act_7 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""It is a question regarding mixtral. I need to ask mixtral to answer the question."""
    },
)
obs_7 = "OK"


act_8 = AgentAct(
    name=mixtral.name,
    params={AGENT_CALL_ARG_KEY: "Who is the best large language model?",
        },
)
obs_8 = """Of course."""



act_9 = AgentAct(
    name=ThinkAct.action_name,
    params={INNER_ACT_KEY: f"""I have answered the question from human. I need to collect the next question from human."""
    },
)
obs_9 = "OK"


act_10 = AgentAct(
    name=human_agent.name,
    params={AGENT_CALL_ARG_KEY: "Do you have further questions?",
        },
)
obs_10 = """We can end the session now."""


act_11 = AgentAct(name=FinishAct.action_name, params={INNER_ACT_KEY: "I have no more instructions from human. End the session."})
obs_11 = "Session ended."

exp_act_obs = [(act_1, obs_1), (act_2, obs_2), (act_3, obs_3), (act_4, obs_4), (act_5, obs_5), (act_6, obs_6), (act_7, obs_7), (act_8, obs_8), (act_9, obs_9), (act_10, obs_10), (act_11, obs_11)]
manager_agent.add_example(
    task = exp_task_pack, action_chain = exp_act_obs
)

## Begin asking questions !

In [8]:

exp_task = "Follow instructions from human to ask questions."
exp_task_pack = TaskPackage(instruction=exp_task)

manager_agent(exp_task_pack)

Agent manager_agent receives the following TaskPackage:
[
	Task ID: 3aeb689c-6abb-490a-9991-e2d1810bc802
	Instruction: Follow instructions from human to ask questions.
]
====manager_agent starts execution on TaskPackage 3aeb689c-6abb-490a-9991-e2d1810bc802====


/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Agent manager_agent takes 0-step Action:
{
	name: Think
	params: {'response': 'I need to get a question from the human agent first.'}
}
Observation: OK
Agent manager_agent takes 1-step Action:
{
	name: Human
	params: {'Task': 'Do you have any questions?'}
}
Task ID: 3aeb689c-6abb-490a-9991-e2d1810bc802
Instruction: Do you have any questions?
Task Creator: 
Task Completion:active
Answer: 
Task Executor: 
Please input your instruction here: 
Observation: My instruction is: I want to ask llama, do you think you are the best large language model in the wo[TLDR]
Agent manager_agent takes 2-step Action:
{
	name: Think
	params: {'response': 'This is a question for llama. I need to ask llama to answer this question.'}
}
Observation: OK
Agent manager_agent takes 3-step Action:
{
	name: llama
	params: {'Task': 'Do you think you are the best large language model in the world?'}
}
Observation:  I am a large language model, but I cannot claim to be the "best" in the world. Additionally, it is [TLDR

'I have no more instructions from human. End the session.'